<a href="https://colab.research.google.com/github/bigflyy/DeepLearningWithPython/blob/main/1NNfromscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import tensorflow as tf
import math
from tensorflow.keras import optimizers
import numpy as np

In [25]:
class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)
  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)
  @property
  def weights(self):
    return [self.W, self.b]

In [26]:
class NaiveSequential:
  def __init__ (self, layers):
    self.layers = layers
  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x
  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights+= layer.weights
    return weights

In [28]:
model = NaiveSequential(
    [NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10,activation=tf.nn.softmax)])

In [29]:
assert len(model.weights) == 4

In [31]:
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert(len(images)== len(labels))
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images)/batch_size)
  def next(self):
    images = self.images[self.index : self.index+self.batch_size]
    labels = self.labels[self.index : self.index+self.batch_size]
    self.index+=self.batch_size
    return images, labels

In [37]:
def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)
  gradients = tape.gradient(average_loss, model.weights)
  update_weights(gradients, model.weights)
  return average_loss

# learning_rate = 1e-3
# def update_weights(gradients, weights):
#   for g,w in zip(gradients, weights):
#     w.assign_sub(g*learning_rate)
from tensorflow.keras import optimizers
optimizer = optimizers.SGD(learning_rate = 1e-3)
def update_weights(gradients, weights):
  optimizer.apply_gradients(zip(gradients, weights))

In [38]:
def fit(model, images, labels, epochs, batch_size=128):
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"loss at batch {batch_counter}: {loss:.2f}")

In [39]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 6.34
loss at batch 100: 2.29
loss at batch 200: 2.26
loss at batch 300: 2.13
loss at batch 400: 2.32
Epoch 1
loss at batch 0: 1.96
loss at batch 100: 1.92
loss at batch 200: 1.87
loss at batch 300: 1.76
loss at batch 400: 1.92
Epoch 2
loss at batch 0: 1.62
loss at batch 100: 1.62
loss at batch 200: 1.54
loss at batch 300: 1.45
loss at batch 400: 1.58
Epoch 3
loss at batch 0: 1.35
loss at batch 100: 1.37
loss at batch 200: 1.26
loss at batch 300: 1.23
loss at batch 400: 1.32
Epoch 4
loss at batch 0: 1.15
loss at batch 100: 1.18
loss at batch 200: 1.06
loss at batch 300: 1.06
loss at batch 400: 1.14
Epoch 5
loss at batch 0: 0.99
loss at batch 100: 1.04
loss at batch 200: 0.92
loss at batch 300: 0.94
loss at batch 400: 1.01
Epoch 6
loss at batch 0: 0.88
loss at batch 100: 0.93
loss at batch 200: 0.81
loss at batch 300: 0.85
loss at batch 400: 0.91
Epoch 7
loss at batch 0: 0.80
loss at batch 100: 0.84
loss at batch 200: 0.73
loss at batch 300: 0.78
loss at batch 40

In [41]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
matches

array([ True,  True,  True, ..., False, False,  True])

In [42]:
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.82


What kind of magic is that?? Just random 512 neurons and some weird linear algebra functions....

In [ ]:
0.82